In [209]:
import re
import pandas as pd

In [210]:
with open('WhatsApp Chat with EEC25.txt', 'r', encoding='utf-8') as f:
    data = f.read()

In [211]:
# Define a function to convert time format to 24-hour format
def convert_to_24_hour(match):
    time_str = match.group()
    time_obj = re.match(r'(\d{1,2}):(\d{2})\s([ap]m)', time_str)
    if time_obj:
        hour, minute, period = time_obj.groups()
        hour = int(hour)
        minute = int(minute)
        if period.lower() == 'pm' and hour != 12:
            hour += 12
        elif period.lower() == 'am' and hour == 12:
            hour = 0
        return f'{hour:02d}:{minute:02d}'
    return time_str

In [212]:
# Use regex to find and replace the time format
data_modified = re.sub(r'\d{1,2}:\d{2}\s[ap]m', convert_to_24_hour, data)
data_modified

'09/02/2022, 18:01 - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.\n09/02/2022, 17:55 - Abhinav Garg EEC created group "EEC\'25🔥"\n09/02/2022, 18:01 - You joined using this group\'s invite link\n09/02/2022, 18:01 - Piyush Jha EEC joined using this group\'s invite link\n09/02/2022, 18:02 - +91 79835 63534 joined using this group\'s invite link\n09/02/2022, 18:02 - Daman joined using this group\'s invite link\n09/02/2022, 18:02 - Labhansh EEC joined using this group\'s invite link\n09/02/2022, 18:02 - Priyank joined using this group\'s invite link\n09/02/2022, 18:03 - +91 62839 01640 joined using this group\'s invite link\n09/02/2022, 18:04 - Kaumudi COE joined using this group\'s invite link\n09/02/2022, 18:04 - +91 77172 84805 joined using this group\'s invite link\n09/02/2022, 18:05 - Saksham EM2: Bahar ka banda na bula lena koi yaha\n09/02/2022, 18:05 - +91 6284 173 339 joined using this gro

In [213]:
# Define the regex pattern to extract messages and dates
pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [214]:
# Split the modified data to extract messages and dates
messages = re.split(pattern, data_modified)[1:]
dates = re.findall(pattern, data_modified)

In [215]:
# Create a DataFrame to store the messages and dates
df = pd.DataFrame({'user_message': messages, 'message_date': dates})

In [216]:
# Convert message_date to datetime format
df['message_date'] = pd.to_datetime(df['message_date'], format='%d/%m/%Y, %H:%M - ')

In [217]:
df = pd.DataFrame({'user_message': messages, 'message_date': dates})
    # convert message_date type
df['message_date'] = pd.to_datetime(df['message_date'], format='%d/%m/%Y, %H:%M - ')

df.rename(columns={'message_date': 'date'}, inplace=True)

In [218]:
df.head()

,user_message,date
0,Messages and calls are end-to-end encrypted. N...,2022-02-09 18:01:00
1,"Abhinav Garg EEC created group ""EEC'25🔥""\n",2022-02-09 17:55:00
2,You joined using this group's invite link\n,2022-02-09 18:01:00
3,Piyush Jha EEC joined using this group's invit...,2022-02-09 18:01:00
4,+91 79835 63534 joined using this group's invi...,2022-02-09 18:02:00


In [219]:
users = []
messages = []
for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s', message)
    if entry[1:]:  # user name
        users.append(entry[1])
        messages.append(" ".join(entry[2:]))
    else:
        users.append('group_notification')
        messages.append(entry[0])

df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)


In [220]:
df.head()

,date,user,message
0,2022-02-09 18:01:00,group_notification,Messages and calls are end-to-end encrypted. N...
1,2022-02-09 17:55:00,group_notification,"Abhinav Garg EEC created group ""EEC'25🔥""\n"
2,2022-02-09 18:01:00,group_notification,You joined using this group's invite link\n
3,2022-02-09 18:01:00,group_notification,Piyush Jha EEC joined using this group's invit...
4,2022-02-09 18:02:00,group_notification,+91 79835 63534 joined using this group's invi...


In [221]:
df['only_date'] = df['date'].dt.date
df['year'] = df['date'].dt.year
df['month_num'] = df['date'].dt.month
df['month'] = df['date'].dt.month_name()
df['day'] = df['date'].dt.day
df['day_name'] = df['date'].dt.day_name()
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

In [222]:
df.head()

,date,user,message,only_date,year,month_num,month,day,day_name,hour,minute
0,2022-02-09 18:01:00,group_notification,Messages and calls are end-to-end encrypted. N...,2022-02-09,2022,2,February,9,Wednesday,18,1
1,2022-02-09 17:55:00,group_notification,"Abhinav Garg EEC created group ""EEC'25🔥""\n",2022-02-09,2022,2,February,9,Wednesday,17,55
2,2022-02-09 18:01:00,group_notification,You joined using this group's invite link\n,2022-02-09,2022,2,February,9,Wednesday,18,1
3,2022-02-09 18:01:00,group_notification,Piyush Jha EEC joined using this group's invit...,2022-02-09,2022,2,February,9,Wednesday,18,1
4,2022-02-09 18:02:00,group_notification,+91 79835 63534 joined using this group's invi...,2022-02-09,2022,2,February,9,Wednesday,18,2
